In [ ]:
import numpy as np
import os
import cv2
import tensorflow as tf
import matplotlib.pyplot as plt
from skimage import io
from glob import glob
from tqdm import tqdm

In [ ]:
image_path='dataset/images/'
mask_path='dataset/masks/'

In [ ]:
train_images=[]
def load_images(path):
    images=glob(os.path.join(path,'*.jpg'))
    for i in tqdm(images):
        i=cv2.imread(i)
        i=cv2.resize(i, (128,128))
        i=cv2.normalize(i,None,0,1,cv2.NORM_MINMAX,cv2.CV_32F)
        train_images.append(i)
    return train_images

train_images=load_images(image_path)

In [ ]:
train_masks=[]
def load_images(path):
    images=glob(os.path.join(path,'*.png'))
    for i in tqdm(images):
        i=cv2.imread(i)
        i=cv2.resize(i, (128,128))
        i=cv2.normalize(i,None,0,1,cv2.NORM_MINMAX,cv2.CV_32F)
        train_masks.append(i)
    return train_masks

train_masks=load_images(mask_path)

In [ ]:
plt.imshow(train_images[0])

In [ ]:
plt.imshow(train_masks[0])

In [ ]:
train_images = np.array(train_images)
train_masks = np.array(train_masks)

In [ ]:
from sklearn.model_selection import train_test_split

trainx, testx, trainy, testy = train_test_split(train_images,train_masks, test_size=0.2, random_state=40)
print(trainx.shape, testx.shape, trainy.shape, testy.shape)

In [ ]:
%env SM_FRAMEWORK=tf.keras
from segmentation_models import Unet, FPN
from segmentation_models import get_preprocessing
from segmentation_models.losses import bce_jaccard_loss
from segmentation_models.metrics import iou_score

from tensorflow.keras.layers import Input, Conv2D
from tensorflow.keras.models import Model

In [ ]:
BACKBONE = 'resnet34'
preprocess_input = get_preprocessing(BACKBONE)

In [ ]:
trainx = preprocess_input(np.array(trainx))
testx = preprocess_input(np.array(testx))

In [ ]:
plt.imshow(trainx[1])

In [ ]:
plt.imshow(testx[1])

In [ ]:
N = train_images.shape[-1]

base_model = FPN(BACKBONE, encoder_weights='imagenet', encoder_freeze=True)
inp = Input(shape=(128,128,N))
l1 = Conv2D(3,(1,1))(inp)
out = base_model(l1)
l2 = Conv2D(3,(1,1))(out)
model = Model(inp,l2,name=base_model.name)
model.summary()

In [ ]:
import tensorflow as tf

from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import binary_crossentropy

In [ ]:
def dice_coefficient(y_true,y_pred):
    numerator = 2*tf.reduce_sum(y_true*y_pred)
    denominator = tf.reduce_sum(y_true+y_pred)
    return numerator / (denominator + tf.keras.backend.epsilon())

In [ ]:
def loss(y_true,y_pred):
    return binary_crossentropy(y_true,y_pred) - tf.math.log(dice_coefficient(y_true,y_pred) + tf.keras.backend.epsilon())

In [ ]:
#model.compile('Adam', loss=loss, metrics=[dice_coefficient])
#model.compile('Adam', loss=bce_jaccard_loss, metrics=[iou_score])
model.compile('Adam', loss = 'binary_crossentropy', metrics=[dice_coefficient])

In [ ]:
model.fit(trainx, trainy, batch_size=8, epochs = 90, validation_data = (testx, testy))

In [ ]:
model.evaluate(testx,testy,verbose=1)

In [ ]:
out = model.predict(np.expand_dims(testx[1],axis=0))[0]
#image = cv2.normalize(out,None,0,1,cv2.NORM_MINMAX,cv2.CV_32F)
plt.imshow((out * 255).astype(np.uint8))

In [ ]:
plt.imshow(testy[1])